In [57]:
import numpy as np
from scipy.optimize import linprog

In [58]:
a = np.array([320, 250, 330])
b = np.array([160, 120, 210, 230, 180])
c = np.array([[13, 8, 15, 10, 7], 
              [6, 18, 12, 9, 4], 
              [14, 11, 16, 17, 25]])

In [59]:
sum(a)

900

In [60]:
sum(b)

900

In [61]:
sum(a) == sum(b) #Суммы равны, следовательно это задача закрытого типа

True

In [62]:
# Необходима функция нахождения индексов минимального элемента матрицы
def minInd(minC):
    c = np.inf
    for i in range(minC.shape[0]):
        for j in range(minC.shape[1]):
            if (minC[i, j] !=0) and (minC[i, j]<c):
                c = minC[i, j]
                i1, j1 = i, j
    return i1, j1

In [63]:
# Функция минимального элемента
def minElement(a, b, c, print = False):
    a = np.copy(a)
    b = np.copy(b)
    c = np.copy(c)        
    m = len(a)
    n = len(b)
    x = np.zeros((m, n), dtype=int) #матрица для x
    f = 0
    while True:
        minC = np.zeros((m,n))
        for i in range(m):
            for j in range(n):
                minC[i, j] = (c[i, j]*min(a[i], b[j])) #матрица суммарных расходов
        i, j = minInd(minC) #индексы мин.элемента
        Xij = int(min(a[i], b[j]))
        x[i, j] = Xij 
        f += int(minC[i, j]) 
        a[i] -= Xij 
        b[j] -= Xij 
        if print:
            print('minC:')
            print(minC.astype(int))
            print('a: ', a)
            print('b: ', b)
            print()
        if len(minC[minC>0])==1: 
            break
    return x, f 

Метод Северо-западного угла

In [64]:
def northwest(a, b, c):
    a = np.copy(a)
    b = np.copy(b)
    c = np.copy(c)
    m = len(a)
    n = len(b)
    i = 0
    j = 0
    f = 0
    x = np.zeros((m, n), dtype=int)
    while (i<m) and (j<n): 
        Xij = min(a[i], b[j]) 
        f += c[i, j]*min(a[i], b[j]) 
        a[i] -= Xij 
        b[j] -= Xij 
        x[i, j] = Xij 
    
        if a[i]>b[j]: 
            j += 1
        elif a[i]<b[j]:
            i += 1
        else:
            i += 1
            j += 1
    return x, f 

Метод потенциалов

In [65]:
def delta(a, b, c, x): 
    m = len(a)
    n = len(b)
    u = np.zeros(m)
    v = np.zeros(n)

    for i in range(m):
        for j in range(n):
            if x[i, j] != 0: 
                if v[j] != 0:
                    u[i] = c[i, j]-v[j]
                else:
                    v[j] = c[i, j]-u[i]

    delta = np.zeros((m, n))
    for i in range(m):
        for j in range(n):
            delta[i, j] = u[i] + v[j] - c[i, j] 
    return delta

In [70]:
# Функция возвращает матрицу системы ограничений
def pre(a, b):
    m = len(a)
    n = len(b)
    u = np.diag(np.ones(n))
    v = np.zeros((m, n))
    v[0] = 1
    for i in range(1, m):
        u = np.hstack((u, np.diag(np.ones(n))))
        k = np.zeros((m, n))
        k[i]=1
        v = np.hstack((v, k))
    return np.vstack((u, v)).astype(int), np.hstack((b,a))

# Метод потенциалов
def potentials(a, b, c):
    a = np.copy(a)
    b = np.copy(b)
    c = np.copy(c)
    m = len(a)
    n = len(b)
    A_eq, b_eq = pre(a, b)
    res = linprog(c.reshape(1, -1), A_eq=A_eq, b_eq=b_eq, bounds=(0, None), method='simplex')
    return res.x.astype(int).reshape(m, n), res.fun.astype(int) # возращаем матрицу x и целевую функцию

In [71]:
x, f = minElement(a, b, c)
print('Метод минимального элемента: \n', x)
print('Целевая функция: ', f)
print()
print('Дельта матрица для М - метода: \n', delta(a, b, c, x))

Метод минимального элемента: 
 [[ 90 120   0 110   0]
 [ 70   0   0   0 180]
 [  0   0 210 120   0]]
Целевая функция:  9770

Дельта матрица для М - метода: 
 [[  0.   0.   1.   0.   4.]
 [  0. -17.  -3.  -6.   0.]
 [  6.   4.   7.   0.  -7.]]


In [72]:
x1, f1 = northwest(a, b, c)
print('Метод северо-западного угла: \n', x1)
print('Целевая функция: ', f1)
print()
print('Дельта матрица для метода северо-западного угла: \n', delta(a, b, c, x1))

Метод северо-западного угла: 
 [[160 120  40   0   0]
 [  0   0 170  80   0]
 [  0   0   0 150 180]]
Целевая функция:  13450

Дельта матрица для метода северо-западного угла: 
 [[  0.   0.   0.   2.  13.]
 [  4. -13.   0.   0.  13.]
 [  4.   2.   4.   0.   0.]]


In [73]:
x2, funk2 = potentials(a, b, c)
print('Метод потенциалов: \n', x2)
print('Целевая функция: ', funk2)
print()
print('Дельта матрица для метода потенциалов: \n', delta(a, b, c, x2))

Метод потенциалов: 
 [[  0   0   0 230  90]
 [160   0   0   0  90]
 [  0 120 210   0   0]]
Целевая функция:  8930

Дельта матрица для метода потенциалов: 
 [[ -7.   3.   1.   0.   0.]
 [ -3. -10.   1.  -2.   0.]
 [ -8.   0.   0.  -7. -18.]]


C:\Users\anyut\AppData\Local\Temp/ipykernel_3296/361978885.py:23: OptimizeWarning: A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.
  res = linprog(c.reshape(1, -1), A_eq=A_eq, b_eq=b_eq, bounds=(0, None), method='simplex')
